In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Custom Model Training

In [ ]:
%pip install -U transformers
%pip install -U accelerate
%pip install -U datasets
%pip install -U bertviz
%pip install -U umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.5 MB/s eta 0:00:00
     ━━

In [ ]:
%pip install spacy
%python -m spacy download en_core_web_sm
%pip install beautifulsoup4
%pip install textblob
%pip install mlxtend
%pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

2024-01-05 10:57:55.446686: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 10:57:55.446740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 10:57:55.448408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 10:57:57.181980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
  Cloning https://github.com/laxmimerit/preprocess_kgptalki

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.autonotebook import tqdm as notebook_tqdm

In [ ]:

class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding="max_length",
                              max_length=self.max_len)

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased'
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sentiData = pd.read_excel('/content/drive/MyDrive/SentimentData/TransformedFinalData.xlsx')
sentiData.columns

Index(['Unnamed: 0', 'Review', 'sentiment'], dtype='object')

In [ ]:
X = sentiData['Review'].tolist()

y = sentiData['sentiment'].tolist()

dataset = CustomDataset(X, y, tokenizer)

In [ ]:
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(example):
  labels = example.label_ids
  preds = example.predictions.argmax(-1)

  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)

  return {'accuracy': acc, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 16
model_name = "distilbert_finetuned_setiment"

args = TrainingArguments(
    output_dir = "output",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size = batch_size,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    evaluation_strategy = 'epoch'
)

In [ ]:
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics=compute_metrics,
                  tokenizer = tokenizer)

In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.189500,0.191219,0.924744,0.924745


TrainOutput(global_step=14381, training_loss=0.21816878189141026, metrics={'train_runtime': 11190.4495, 'train_samples_per_second': 20.561, 'train_steps_per_second': 1.285, 'total_flos': 3.047968869155635e+16, 'train_loss': 0.21816878189141026, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/SentimentData/distilbertModel")

In [ ]:
from transformers import DistilBertConfig, DistilBertModel
path = '/content/drive/MyDrive/SentimentData/distilbertModel'
model = DistilBertModel.from_pretrained(path)

In [ ]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertModel

In [ ]:
trainer.save_model("/content/drive/MyDrive/SentimentData/"+model_name)

In [ ]:
from transformers import pipeline
text = "i love this product"
pipe = pipeline('text-classification', model_name)
pipe(text)

[{'label': 'LABEL_1', 'score': 0.9651976227760315}]

In [ ]:
tok = AutoTokenizer.from_pretrained(model_name)
mod = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
def get_prediction(text):
  input_ids = tok.encode(text, return_tensors='pt')
  print("input_ids",input_ids)
  output = mod(input_ids)
  print("output:",output)
  preds = torch.nn.functional.softmax(output.logits, dim=-1)
  print("preds:",preds)
  prob = torch.max(preds).item()
  print("prob:",prob)
  idx = torch.argmax(preds).item()
  # sentiment = id2label[idx]

  return {'sentiment':idx, 'prob':prob}

In [ ]:
text = "The food was worst and fabulous"
get_prediction(text)

input_ids tensor([[  101,  1996,  2833,  2001,  5409,  1998, 18783,   102]])
output: SequenceClassifierOutput(loss=None, logits=tensor([[ 1.4579, -1.3158]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
preds: tensor([[0.9412, 0.0588]], grad_fn=<SoftmaxBackward0>)
prob: 0.9412362575531006


{'sentiment': 0, 'prob': 0.9412362575531006}

In [1]:
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI\\research'

In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI'

In [4]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

c:\Users\akish\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
name = 'distilbert_finetuned_setiment'
tok = AutoTokenizer.from_pretrained(name)
mod = AutoModelForSequenceClassification.from_pretrained(name)

RuntimeError: Failed to import transformers.models.distilbert.modeling_distilbert because of the following error (look up to see its traceback):
No module named 'torch.cuda.amp'

In [12]:
def get_prediction(text):
  input_ids = tok.encode(text, return_tensors='pt')
  print("input_ids",input_ids)
  output = mod(input_ids)
  print("output:",output)
  preds = torch.nn.functional.softmax(output.logits, dim=-1)
  print("preds:",preds)
  prob = torch.max(preds).item()
  print("prob:",prob)
  idx = torch.argmax(preds).item()
  # sentiment = id2label[idx]

  return {'sentiment':idx, 'prob':prob}

In [13]:
text = "The food was worst and fabulous"
get_prediction(text)

input_ids tensor([[  101,  1996,  2833,  2001,  5409,  1998, 18783,   102]])
output: SequenceClassifierOutput(loss=None, logits=tensor([[ 1.4579, -1.3158]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
preds: tensor([[0.9412, 0.0588]], grad_fn=<SoftmaxBackward0>)
prob: 0.941236138343811


{'sentiment': 0, 'prob': 0.941236138343811}